In [1]:
import json
import pandas as pd
import re

from scrape_prydwen import scrape_prydwen

In [2]:
version = '2.6.0'

In [3]:
def normalize_df(df, col_name):
    normalized_data = pd.json_normalize(df[col_name]).add_prefix(f'{col_name}.')
    df = pd.concat([df, normalized_data], axis=1)
    df = df.drop(col_name, axis=1)
    return df

In [4]:
scrape_prydwen(version)

2026-02-15 12:54:47.233228: Started processing
Now processing: https://www.prydwen.gg/zenless/characters/alice
Finished processing: https://www.prydwen.gg/zenless/characters/alice
Now processing: https://www.prydwen.gg/zenless/characters/anby-demara
Finished processing: https://www.prydwen.gg/zenless/characters/anby-demara
Now processing: https://www.prydwen.gg/zenless/characters/anby-demara-soldier-0
Finished processing: https://www.prydwen.gg/zenless/characters/anby-demara-soldier-0
Now processing: https://www.prydwen.gg/zenless/characters/anton
Finished processing: https://www.prydwen.gg/zenless/characters/anton
Now processing: https://www.prydwen.gg/zenless/characters/aria
Could not find Drive Discs section
Finished processing: https://www.prydwen.gg/zenless/characters/aria
Now processing: https://www.prydwen.gg/zenless/characters/astra-yao
Finished processing: https://www.prydwen.gg/zenless/characters/astra-yao
Now processing: https://www.prydwen.gg/zenless/characters/banyue
Finis

In [5]:
file_path = f'characters_output_{version}.json'
json_file = open(file_path)
json_dict = json.load(json_file)

df = pd.DataFrame(json_dict)

df = df.transpose()
df = df.reset_index(drop=True)

In [6]:
df = normalize_df(df, 'Drive Discs')

# Rename Drive Disc columns for clarity
df.columns = df.columns.str.replace(r'Drive Discs\.(\d+)\.4pc Set', r'4pc_\1', regex=True)
df.columns = df.columns.str.replace(r'Drive Discs\.(\d+)\.Recommended 2pc Sets', r'Recommended 2pc_\1', regex=True)
df.columns = df.columns.str.replace(r'Drive Discs\.(\d+)\.All 2pc Sets', r'All 2pc_\1', regex=True)

In [7]:
df = normalize_df(df, 'Stats')

df.rename(columns={
    'Stats.Disc 4': 'Disc 4',
    'Stats.Disc 5': 'Disc 5',
    'Stats.Disc 6': 'Disc 6',
    'Stats.Substats': 'Substats'
}, inplace=True)

In [8]:
# Filter out agents with empty builds
df = df.dropna(subset=['4pc_0'])

# Add computed columns containing combinations of 4pc and 2pc sets
columns = df.columns.tolist()
four_pc_columns = [col for col in columns if re.search(r'4pc_\d+$', col)]
for i, four_pc_column in enumerate(four_pc_columns):
    df[f'4pc w/ Recommended 2pc_{i}'] = df[four_pc_column] + ', ' + df[f'Recommended 2pc_{i}']
    df[f'4pc w/ All 2pc_{i}'] = df[four_pc_column] + ', ' + df[f'All 2pc_{i}']

# Add new columns for concatenating the top sets
df['Final Set Concat'] = """=SWITCH('Set+Main'!$L$3, "Recommended", INDIRECT(ADDRESS(ROW(), 7)), "All", INDIRECT(ADDRESS(ROW(), 12)), "None", INDIRECT(ADDRESS(ROW(), 17)))"""
df['4pc w/ Recommended 2pc Concat'] = """=TEXTJOIN(", ", TRUE, INDIRECT(ADDRESS(ROW(), COLUMN() + 1)):INDIRECT(ADDRESS(ROW(), COLUMN() + 'Set+Main'!$L$2)))"""
df['4pc w/ All 2pc Concat'] = """=TEXTJOIN(", ", TRUE, INDIRECT(ADDRESS(ROW(), COLUMN() + 1)):INDIRECT(ADDRESS(ROW(), COLUMN() + 'Set+Main'!$L$2)))"""
df['4pc Concat'] = """=TEXTJOIN(", ", TRUE, INDIRECT(ADDRESS(ROW(), COLUMN() + 1)):INDIRECT(ADDRESS(ROW(), COLUMN() + 'Set+Main'!$L$2)))"""

In [9]:
# Reorder columns
columns = df.columns.tolist()
four_pc_with_recommended_two_pc_columns = [col for col in columns if re.search(r'4pc w/ Recommended 2pc_\d+', col)]
four_pc_with_all_two_pc_columns = [col for col in columns if re.search(r'4pc w/ All 2pc_\d+', col)]
recommended_two_pc_columns = [col for col in columns if re.search(r'^Recommended 2pc_\d+', col)]
all_two_pc_columns = [col for col in columns if re.search(r'^All 2pc_\d+', col)]

column_order = ['Name', 'Disc 4', 'Disc 5', 'Disc 6', 'Substats', 'Final Set Concat', '4pc w/ Recommended 2pc Concat']
column_order.extend(four_pc_with_recommended_two_pc_columns)
column_order.append('4pc w/ All 2pc Concat')
column_order.extend(four_pc_with_all_two_pc_columns)
column_order.append('4pc Concat')
column_order.extend(four_pc_columns)
column_order.extend(recommended_two_pc_columns)
column_order.extend(all_two_pc_columns)

df = df[column_order]

In [10]:
# Preview dataframe before exporting to csv
df

,Name,Disc 4,Disc 5,Disc 6,Substats,Final Set Concat,4pc w/ Recommended 2pc Concat,4pc w/ Recommended 2pc_0,4pc w/ Recommended 2pc_1,4pc w/ Recommended 2pc_2,...,4pc_2,4pc_3,Recommended 2pc_0,Recommended 2pc_1,Recommended 2pc_2,Recommended 2pc_3,All 2pc_0,All 2pc_1,All 2pc_2,All 2pc_3
0,Alice,Anomaly Proficiency > ATK%,PEN Ratio% > Physical DMG% > ATK%,Anomaly Mastery,Anomaly Proficiency > ATK% > PEN > ATK,"=SWITCH('Set+Main'!$L$3, ""Recommended"", INDIRE...","=TEXTJOIN("", "", TRUE, INDIRECT(ADDRESS(ROW(), ...","Fanged Metal, Phaethon's Melody","Hormone Punk, Phaethon's Melody",NaN,...,NaN,NaN,Phaethon's Melody,Phaethon's Melody,NaN,NaN,"Phaethon's Melody, Puffer Electro, Freedom Blu...","Phaethon's Melody, Puffer Electro, Freedom Blu...",NaN,NaN
1,Anby,CRIT Rate% >= CRIT DMG%,Electric DMG% > ATK%,Impact,CRIT RATE = CRIT DMG > ATK% > PEN = ATK,"=SWITCH('Set+Main'!$L$3, ""Recommended"", INDIRE...","=TEXTJOIN("", "", TRUE, INDIRECT(ADDRESS(ROW(), ...","King of the Summit, Shockstar Disco","Astral Voice, King of the Summit","Proto Punk, King of the Summit",...,Proto Punk,Shockstar Disco,Shockstar Disco,King of the Summit,King of the Summit,King of the Summit,"Shockstar Disco, Swing Jazz","King of the Summit, Shockstar Disco, Swing Jazz","King of the Summit, Shockstar Disco, Swing Jazz","King of the Summit, Swing Jazz"
2,Anby: Soldier 0,CRIT Rate% >= CRIT DMG%,Electric DMG% > ATK%,ATK%,CRIT RATE = CRIT DMG > ATK% > PEN = ATK,"=SWITCH('Set+Main'!$L$3, ""Recommended"", INDIRE...","=TEXTJOIN("", "", TRUE, INDIRECT(ADDRESS(ROW(), ...","Shadow Harmony, Woodpecker Electro",NaN,NaN,...,NaN,NaN,Woodpecker Electro,NaN,NaN,NaN,"Woodpecker Electro, Hormone Punk, Astral Voice...",NaN,NaN,NaN
3,Anton,CRIT Rate% >= CRIT DMG%,Electric DMG%,ATK%,CRIT RATE = CRIT DMG > ATK% > PEN = ATK,"=SWITCH('Set+Main'!$L$3, ""Recommended"", INDIRE...","=TEXTJOIN("", "", TRUE, INDIRECT(ADDRESS(ROW(), ...","Thunder Metal, Woodpecker Electro","Puffer Electro, Woodpecker Electro","Woodpecker Electro, Puffer Electro",...,Woodpecker Electro,NaN,Woodpecker Electro,Woodpecker Electro,Puffer Electro,NaN,"Woodpecker Electro, Hormone Punk, Puffer Electro","Woodpecker Electro, Hormone Punk, Thunder Metal","Puffer Electro, Hormone Punk, Thunder Metal",NaN
5,Astra Yao,ATK%,ATK%,Energy Regen = ATK%,ATK% > Flat ATK (Top Priorities Until Ability ...,"=SWITCH('Set+Main'!$L$3, ""Recommended"", INDIRE...","=TEXTJOIN("", "", TRUE, INDIRECT(ADDRESS(ROW(), ...","Astral Voice, Swing Jazz, Moonlight Lullaby","Moonlight Lullaby, Swing Jazz",NaN,...,NaN,NaN,"Swing Jazz, Moonlight Lullaby",Swing Jazz,NaN,NaN,"Swing Jazz, Moonlight Lullaby, Hormone Punk","Swing Jazz, Hormone Punk, Astral Voice",NaN,NaN
6,Banyue,CRIT DMG% > CRIT Rate%,Fire DMG% > HP%,HP%,CRIT Rate = CRIT DMG > HP% > ATK% > Flat HP,"=SWITCH('Set+Main'!$L$3, ""Recommended"", INDIRE...","=TEXTJOIN("", "", TRUE, INDIRECT(ADDRESS(ROW(), ...","Yunkui Tales, Woodpecker Electro",NaN,NaN,...,NaN,NaN,Woodpecker Electro,NaN,NaN,NaN,"Woodpecker Electro, Branch & Blade Song",NaN,NaN,NaN
7,Ben,DEF% > CRIT DMG%,DEF% = Fire DMG%,DEF% = Energy Regen,DEF% > CRIT RATE% (Optional) = CRIT DMG% (Opti...,"=SWITCH('Set+Main'!$L$3, ""Recommended"", INDIRE...","=TEXTJOIN("", "", TRUE, INDIRECT(ADDRESS(ROW(), ...","Swing Jazz, Soul Rock","Proto Punk, Swing Jazz","Shockstar Disco, Soul Rock",...,Shockstar Disco,NaN,Soul Rock,Swing Jazz,Soul Rock,NaN,"Soul Rock, Puffer Electro, Inferno Metal","Swing Jazz, Shockstar Disco","Soul Rock, Swing Jazz, Freedom Blues, Inferno ...",NaN
8,Billy,CRIT Rate% = CRIT DMG%,Physical DMG% >= ATK%,ATK%,CRIT RATE = CRIT DMG > ATK% > PEN = ATK,"=SWITCH('Set+Main'!$L$3, ""Recommended"", INDIRE...","=TEXTJOIN("", "", TRUE, INDIRECT(ADDRESS(ROW(), ...","Woodpecker Electro, Puffer Electro","Puffer Electro, Woodpecker Electro","Fanged Metal, Woodpecker Electro",...,Fanged Metal,NaN,Puffer Electro,Woodpecker Electro,Woodpecker Electro,NaN,"Puffer Electro, Hormone Punk, Fanged Metal","Woodpecker Electro, Hormone Punk, Fanged Me

In [11]:
df.to_csv(f'drive_disc_salvaging_helper_{version}.csv', index=False)
print(f'Output saved to drive_disc_salvaging_helper_{version}.csv')

Output saved to drive_disc_salvaging_helper_2.5.0.csv
